In [60]:
# Imports 📥

# Packages
#-----------------------------------------------------------------------
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Visualization
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Settings
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Data 
# -----------------------------------------------------------------------
abc_coorp_df = pd.read_csv("HR_RAW_DATA.csv", index_col=0)
abc_coorp_df.head(2)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",1,0,51,3,5,resEArch DIREcToR,3,NaN,"19537,0",6462,7,Y,No,13,"3,0",3,NaN,0,NaN,5,"3,0",20,NaN,15,15,"19537,0",1972,1000000000$,NaN,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,Life Sciences,1,"259,0",3,0,65,2,5,ManAGeR,3,NaN,"19999,0",5678,0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,"19999,0",1971,1000000000$,NaN,NaN,1


In [50]:
abc_coorp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 0 to 1613
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1614 non-null   object 
 1   Attrition                 1614 non-null   object 
 2   BusinessTravel            842 non-null    object 
 3   DailyRate                 1614 non-null   object 
 4   Department                302 non-null    object 
 5   DistanceFromHome          1614 non-null   int64  
 6   Education                 1614 non-null   int64  
 7   EducationField            869 non-null    object 
 8   employeecount             1614 non-null   int64  
 9   employeenumber            1183 non-null   object 
 10  EnvironmentSatisfaction   1614 non-null   int64  
 11  Gender                    1614 non-null   int64  
 12  HourlyRate                1614 non-null   object 
 13  JobInvolvement            1614 non-null   int64  
 14  JobLevel     

In [55]:
# DataTransformer class definition ✍️

class DataTransformer:
    def __init__(self, dataframe):
        self.df = dataframe

    def replace_gender_values(self):
        """Replaces the values in the Gender column with 'Male' and 'Female'."""
        self.df['Gender'] = self.df['Gender'].replace({0: 'Male', 1: 'Female'})

    def convert_object_to_float_eliminate_dolar(self, column_name):
        """
        Converts a column with object type values containing $ signs at the end
        and commas as decimal separators to numeric float values.
        """
        if column_name in self.df.columns:
            # Remove dollar signs from the end
            self.df[column_name] = self.df[column_name].str.replace('$', '', regex=False)
            
            # Replace commas with dots for decimal points
            self.df[column_name] = self.df[column_name].str.replace(',', '.', regex=False)
            
            # Convert the cleaned column to float type
            self.df[column_name] = self.df[column_name].astype(float)
        else:
            print(f"Column {column_name} doesn't exist in the DataFrame.")
        

    def convert_role_to_department_normalize_job_role(self):
        # First change de type of data to be capitalize and the same way
        self.df['JobRole'] = self.df['JobRole'].str.title()
        self.df['Department'] = self.df['Department'].str.title()
       # Clean empty spaces
        self.df['JobRole'] = self.df['JobRole'].str.strip()
        self.df['Department'] = self.df['Department'].str.strip()
        
        conversion_dictionary = {
        'Healthcare Representative': 'Research & Development',
        'Sales Executive': 'Sales',
        'Healthcare Representative': 'Research & Development',
        'Laboratory Technician': 'Research & Development',
        'Manufacturing Director': 'Research & Development',
        'Research Scientist': 'Research & Development',
        'Sales Executive': 'Sales',
        'Sales Representative':'Sales',
        'Research Director': 'Research & Development',
        'Human Resources': 'Human Resources',
         }
        # Iterate over the rows of the DataFrame
        for index, row in self.df.iterrows():
            job_role = row['JobRole']
            # Assign the corresponding value to Department using the dictionary
            if job_role in conversion_dictionary:
                self.df.at[index, 'Department'] = conversion_dictionary[job_role]
                print(f"Value '{job_role}' was transformed into '{conversion_dictionary[job_role]}'")

    def change_marital_status(self):
        
        self.df['MaritalStatus'] = self.df['MaritalStatus'].replace({
        "Marreid": "Married",
        "divorced": "Divorced"})
        self.df["MaritalStatus"] = self.df["MaritalStatus"].fillna("Unknown")
        
    def map_column_remote_work(self):
        dicc = {1: "True", 0: "False", "Yes": "True"}

        # Replace the values in the RemoteWork column according to the dictionary
        self.df["RemoteWork"] = self.df["RemoteWork"].replace(dicc)


    def change_null_for_unknown(self, column_list): # when doesnt exist a dominant category in categorical variable
        # Iterate through the list of columns to replace nulls with "Unknown"
        for column in column_list:
            if column in self.df.columns:
                # Replace nulls with the value "Unknown" for each column in the list
                self.df[column] = self.df[column].fillna("Unknown")
            else:
                print(f"Warning: The column '{column}' does not exist in the DataFrame.")
        return self.df
    
    def change_null_for_mode(self, column_list): # When we have a dominant category in categorical variables
        for column in column_list:
            if column in self.df.columns:
                # Calculate the mode of the column
                mode = self.df[column].mode()[0]
                # Replace nulls with the mode for each column in the list
                self.df[column] = self.df[column].fillna(mode)
            else:
                print(f"Warning: The column '{column}' does not exist in the DataFrame.")
        return self.df
    
    def change_null_for_mean(self, column_list): # when we have a 0-10% of nulls in numerical category and distribution is normal
        # Iterate through the list of columns to replace nulls with mean
        for column in column_list:
            if column in self.df.columns:
                    mean= self.df[column].mean()
                # Replace nulls with the mode for each column in the list
                    self.df[column] = self.df[column].fillna(mean)
            else:
                print(f"Warning: The column '{column}' does not exist in the DataFrame.")
        return self.df
    
    def change_null_for_median(self, column_list): # when we have a 0-10% of nulls in numerical category and distribution is atypical
        # Iterate through the list of columns to replace nulls with median
        for column in column_list:
            if column in self.df.columns:
                    median= self.df[column].median()
                # Replace nulls with the mode for each column in the list
                    self.df[column] = self.df[column].fillna(median)
            else:
                print(f"Warning: The column '{column}' does not exist in the DataFrame.")
        return self.df
    
    def impute_with_knn(self, column_list, n_neighbors=5): # when we have a numerical variable with more than 10% of nulls
        # Create an instance of KNNImputer
        imputer_knn = KNNImputer(n_neighbors=n_neighbors)

        # Fit and transform the data
        imputed_data = imputer_knn.fit_transform(self.df[column_list])

        # Convert the result to a DataFrame
        imputed_df = pd.DataFrame(imputed_data, columns=column_list)

        # Add the imputed columns to the original DataFrame
        for column in column_list:
            self.df[f"{column}_knn"] = imputed_df[column]

        return self.df

In [56]:
# Instantiating a class object 🐣

abc_data = DataTransformer(abc_coorp_df)

In [ ]:
# ▶️ convert_object_to_float_eliminate_dolar

abc_data.convert_object_to_float_eliminate_dolar()
print(abc_data)

abc_data.df["DailyRate"].sample(50)
abc_data.df["DailyRate"].unique()
abc_data.df["DailyRate"].value_counts()
abc_coorp_df

In [ ]:
# ▶️ Convert_role_to_department and normalize jobrole letters
abc_data.convert_role_to_department_normalize_job_role()
print(abc_data)

abc_data.df["JobRole"].sample(50)
abc_data.df["JobRole"].unique()
abc_data.df["Department"].value_counts()
abc_data.df[["JobRole","Department"]].sample(50)
abc_coorp_df

In [ ]:
# ▶️ Transform "marital_status" data

abc_data.change_marital_status()
abc_data.df["MaritalStatus"].unique()
abc_data.df["MaritalStatus"].value_counts()
abc_data.df[["MaritalStatus"]].sample(50)
abc_coorp_df



In [6]:
# ▶️ Transform "remote work"
abc_data.map_column_remote_work()
abc_coorp_df["RemoteWork"].unique()

abc_coorp_df["RemoteWork"].value_counts()

RemoteWork
True     640
1        360
0        309
False    305
Name: count, dtype: int64

In [57]:
# ▶️ Change nulls for unknown category in categorial variable

# if we want to change more than one column , put a list
columns_modify = []
abc_data.change_null_for_unknown(["Over18"])
abc_coorp_df["Over18"].isnull().sum()

0

In [58]:
# ▶️ Change nulls for mode in categorical variable

# if we want to change more than one column , put a list
columns_modify = []
abc_data.change_null_for_mode(["OverTime"])
abc_coorp_df["OverTime"].isnull().sum()

0

In [59]:
# ▶️ Change nulls for mean in numerical variable when we have a 0-10% of nulls in numerical category and distribution is normal

# if we want to change more than one column , put a list
columns_modify = []
abc_data.change_null_for_mean(["DailyRate"])
abc_coorp_df["DailyRate"].isnull().sum()

TypeError: Could not convert string '684,0$699,0$532,0$359,0$1319,0$117,0$1435,0$635,0$1276,0$840,0$247,0$1369,0$201,0$1360,0$692,0$1398,0$286,0$1402,0$819,0$884,0$1238,0$515,0$1223,0$202,0$928,0$607,0$266,0$429,0$589,0$nan$nan$1180,0$1282,0$776,0$665,0$526,0$1034,0$1403,0$1499,0$580,0$859,0$263,0$1376,0$885,0$1003,0$1321,0$nan$394,0$1372,0$nan$1333,0$228,0$737,0$823,0$667,0$301,0$573,0$nan$1329,0$630,0$nan$1063,0$1017,0$1296,0$939,0$1355,0$1448,0$200,0$1202,0$404,0$208,0$813,0$465,0$1189,0$nan$1001,0$1394,0$161,0$288,0$682,0$1354,0$147,0$119,0$1413,0$452,0$334,0$1132,0$nan$982,0$480,0$1099,0$672,0$1379,0$583,0$1492,0$1050,0$117,0$469,0$nan$237,0$1440,0$1291,0$1157,0$1336,0$1224,0$735,0$nan$1389,0$638,0$1240,0$194,0$1339,0$111,0$nan$1469,0$470,0$1232,0$1249,0$117,0$798,0$672,0$549,0$570,0$1232,0$541,0$164,0$1117,0$619,0$319,0$956,0$1245,0$1397,0$527,0$213,0$nan$527,0$882,0$330,0$406,0$217,0$481,0$669,0$1465,0$685,0$1062,0$1099,0$177,0$1199,0$648,0$412,0$282,0$770,0$601,0$855,0$332,0$13

In [ ]:
# ▶️ Change nulls for median in numerical variable when we have a 0-10% of nulls in numerical category and distribution is atypical

# if we want to change more than one column , put a list
columns_modify = []
abc_data.change_null_for_mean(["column"])
abc_coorp_df["column"].isnull().sum()

In [ ]:
# ▶️ Change nulls for distribution KNN in numerical variable when we have more 10% of nulls in numerical category 

# if we want to change more than one column , put a list
columns_modify = []
abc_data.change_null_for_mean(["column"])
abc_coorp_df["column"].isnull().sum()

In [4]:
# ▶️ Eliminate duplicated "employee_number"

duplicate_employeenumbers = abc_coorp_df['employeenumber'].value_counts()

abc_coorp_df['employeenumber'].isnull().sum()

duplicate_employeenumbers

employeenumber
482,0     2
530,0     2
507,0     2
517,0     2
522,0     2
         ..
161,0     1
164,0     1
190,0     1
194,0     1
2040,0    1
Name: count, Length: 1079, dtype: int64

In [7]:
#employeenumber`: Variable categórica nominal, al tratarse de un identificador único para cada empleado (número). 
# Pero estos números están en formato de texto y posiblemente con un formato de decimal que en realidad no es necesario. 
# Hay 534 duplicados en esta columna. Los valores deben ser unicos.  Hablamos con César. debemos analizar los duplicados mejor 
# y tomar decisiones en base a las conclusiones. Por lo que vimos parece que la diferencia entre los duplicados está en Remote Work, 
# puede que sea un error en la carga de datos o que se haya hecho mas de una vez la encuesta. Explorar mas.

duplicados= abc_coorp_df.loc[abc_coorp_df['employeenumber'].duplicated(keep=False), :].sort_values(by='employeenumber')

    
# Identify duplicates
abc_coorp_df.loc[abc_coorp_df['employeenumber'].duplicated(keep=False), :].sort_values(by='employeenumber').sample(100)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
1555,29,No,travel_rarely,"144,0$",NaN,10,1,NaN,1,NaN,4,1,39,2,2,SAles EXecuTiVe,2,Divorced,NaN,11866,1,Y,Yes,14,"3,0",1,NaN,2,"7,0",2,"3,0",7,NaN,1,7,NaN,1994,1000000000$,NaN,NaN,Yes
526,45,No,NaN,"1316,0$",NaN,29,3,NaN,1,NaN,3,0,83,3,1,rESEArch SciEntIST,4,Single,"3452,0",9752,5,NaN,NaN,13,"3,0",2,NaN,0,"9,0",2,"2,0",6,NaN,0,3,"3452,0",1978,1000000000$,NaN,NaN,1
22,36,No,NaN,"1223,0$",NaN,8,3,NaN,1,NaN,3,1,59,3,3,hEaltHCarE ReprESENTAtIvE,3,Divorced,NaN,8202,1,Y,NaN,13,NaN,2,NaN,3,"17,0",2,"3,0",17,NaN,12,8,NaN,1987,1000000000$,NaN,NaN,1
937,29,No,NaN,"1090,0$",Sales,10,3,NaN,1,NaN,4,0,83,3,1,SaLes rePrESENTaTivE,2,NaN,"2297,0",17967,1,Y,No,14,"3,0",4,"80,0",2,NaN,2,"3,0",2,NaN,2,2,"2297,0",1994,1000000000$,SaLes rePrESENTaTivE - Sales,NaN,0
781,28,No,non-travel,"1103,0$",NaN,16,3,Medical,1,"1947,0",3,0,49,3,1,reSeArch sCieNTiSt,3,Single,"2144,0",2122,1,NaN,No,14,"3,0",3,NaN,0,"5,0",3,"2,0",5,NaN,1,4,"2144,0",1995,1000000000$,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,39,No,NaN,"408,0$",NaN,2,4,Technical Degree,1,"721,0",4,1,80,2,2,HeaLTHcARE reprESenTAtIve,3,Single,NaN,20978,1,NaN,No,11,"3,0",1,NaN,0,"20,0",4,"3,0",20,"7,0",11,10,NaN,1984,1000000000$,NaN,NaN,0
878,37,No,NaN,"1192,0$",NaN,5,2,Medical,1,"460,0",4,0,61,3,2,ManUFaCTURing dIRectOR,4,NaN,"6347,0",23177,7,Y,No,16,"3,0",3,NaN,2,"8,0",2,"2,0",6,NaN,0,4,"6347,0",1986,1000000000$,NaN,NaN,Yes
329,31,No,travel_rarely,"525,0$",NaN,6,4,Medical,1,"653,0",1,0,66,4,2,SALes EXEcUTiVe,4,Divorced,NaN,6219,4,Y,No,22,"4,0",4,NaN,2,"13,0",4,"4,0",7,NaN,5,7,NaN,1992,1000000000$,NaN,NaN,1
1495,35,No,travel_frequently,nan$,NaN,4,4,Other,1,"1185,0",4,0,47,2,1,laboRatORY teCHnICIaN,4,Married,"2376,0",26537,1,NaN,NaN,13,"3,0",2,NaN,1,"2,0",2,"4,0",2,NaN,2,2,"2376,0",1988,1000000000$,NaN,NaN,True


In [6]:
# Al verificar los nulos, comprobamos que el número ímpar se da por esa razón y subtraímos las filas nulas de las filas totales para saber cuantos duplicados hay.
# el resultado de la suma tiene que ser un número par.
duplicados.shape[0] - duplicados['employeenumber'].isnull().sum()

208

In [8]:
# Concluímos que tenemos 208 empleados con el ID de empleado duplicado. 
# Ahora vamos a remover estos nulos de nuestro df de duplicados para poder tratar los duplicados. 
duplicados.dropna(subset='employeenumber', inplace=True)

In [11]:
# Ahora borramos todos los duplicados que tengan el menor indice, ya que suponemos que los de indice mayor son los registros mas actualizados. 
duplicados.index.name = 'index'  # Agregar nombre al índice
duplicados.shape

(208, 41)

In [10]:
# Ordenar el DataFrame por 'employeenumber' y por el índice en orden descendente
df_sorted = duplicados.sort_values(by=['employeenumber', duplicados.index.name], ascending=[True, False])
df_sorted

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1483,33,No,non-travel,"1038,0$",Sales,8,1,Life Sciences,1,"1044,0",2,1,88,2,1,sales repResENTaTiVE,4,NaN,NaN,21437,0,Y,NaN,19,"3,0",4,NaN,0,NaN,2,"2,0",2,NaN,2,2,NaN,1990,1000000000$,sales repResENTaTiVE - Sales,NaN,0
987,33,No,non-travel,"1038,0$",Sales,8,1,Life Sciences,1,"1044,0",2,1,Not Available,2,1,sALES ReprEsentatiVE,4,NaN,NaN,21437,0,Y,NaN,19,"3,0",4,NaN,0,NaN,2,"2,0",2,NaN,2,2,NaN,1990,1000000000$,sALES ReprEsentatiVE - Sales,NaN,False
1484,26,Yes,NaN,"342,0$",NaN,2,3,Life Sciences,1,"1053,0",1,0,57,3,1,reSEArcH SCiEnTIst,1,Married,"2042,0",15346,6,Y,NaN,14,"3,0",2,NaN,1,"6,0",2,"3,0",3,NaN,1,2,"2042,0",1997,1000000000$,NaN,NaN,Yes
988,26,Yes,NaN,"342,0$",NaN,2,3,Life Sciences,1,"1053,0",1,0,57,3,1,rEsEaRCH SCIenTiSt,1,Married,"2042,0",15346,6,Y,NaN,14,"3,0",2,NaN,1,"6,0",2,"3,0",3,NaN,1,2,"2042,0",1997,1000000000$,NaN,NaN,1
1486,46,No,NaN,"430,0$",NaN,1,4,Medical,1,"1069,0",4,0,40,3,5,rESEArCh DIrecTOR,4,NaN,NaN,21445,9,NaN,No,17,"3,0",4,"80,0",2,"23,0",0,"3,0",2,NaN,2,2,NaN,1977,1000000000$,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,36,Yes,travel_rarely,"530,0$",NaN,3,1,Life Sciences,1,"967,0",3,0,51,2,3,SALes EXECuTiVe,4,Married,"10325,0",5518,1,Y,NaN,11,NaN,1,NaN,1,NaN,6,"3,0",16,NaN,3,7,"10325,0",1987,1000000000$,NaN,NaN,0
1610,45,No,non-travel,"805,0$",NaN,4,2,NaN,1,"972,0",3,0,57,3,2,LAboRaTOry tECHNiCIAn,2,NaN,"4447,0",23163,1,NaN,NaN,12,"3,0",2,NaN,0,NaN,5,"2,0",9,NaN,0,8,"4447,0",1978,1000000000$,NaN,NaN,1
367,45,No,non-travel,"805,0$",NaN,-47,2,NaN,1,"972,0",47,0,57,3,2,lABORaTory tecHNiCiAN,2,NaN,"4447,0",23163,1,NaN,NaN,12,"3,0",2,NaN,0,NaN,5,"2,0",9,NaN,0,8,"4447,0",1978,1000000000$,NaN,NaN,1


In [11]:
# Eliminar duplicados conservando el primer registro encontrado (índice más alto)
df_unique = df_sorted.drop_duplicates(subset='employeenumber', keep='first')
df_unique # nos hemos quedado con 104 filas

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1483,33,No,non-travel,"1038,0$",Sales,8,1,Life Sciences,1,"1044,0",2,1,88,2,1,sales repResENTaTiVE,4,NaN,NaN,21437,0,Y,NaN,19,"3,0",4,NaN,0,NaN,2,"2,0",2,NaN,2,2,NaN,1990,1000000000$,sales repResENTaTiVE - Sales,NaN,0
1484,26,Yes,NaN,"342,0$",NaN,2,3,Life Sciences,1,"1053,0",1,0,57,3,1,reSEArcH SCiEnTIst,1,Married,"2042,0",15346,6,Y,NaN,14,"3,0",2,NaN,1,"6,0",2,"3,0",3,NaN,1,2,"2042,0",1997,1000000000$,NaN,NaN,Yes
1486,46,No,NaN,"430,0$",NaN,1,4,Medical,1,"1069,0",4,0,40,3,5,rESEArCh DIrecTOR,4,NaN,NaN,21445,9,NaN,No,17,"3,0",4,"80,0",2,"23,0",0,"3,0",2,NaN,2,2,NaN,1977,1000000000$,NaN,NaN,1
1487,21,No,NaN,"984,0$",NaN,-25,1,NaN,1,"1131,0",4,1,70,2,1,REsEaRch scIenTIsT,2,Single,"2070,0",25326,1,Y,Yes,11,"3,0",3,"80,0",0,"2,0",6,"4,0",2,NaN,2,2,"2070,0",2002,1000000000$,NaN,NaN,1
1488,31,No,travel_frequently,"793,0$",NaN,20,3,NaN,1,"1135,0",3,0,67,4,1,saLES RePRESenTAtIve,4,Married,"2791,0",21981,0,NaN,No,12,NaN,1,NaN,1,"3,0",4,"3,0",2,NaN,2,2,"2791,0",1992,1000000000$,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606,26,No,travel_frequently,"1283,0$",Sales,1,3,Medical,1,"956,0",3,0,52,2,2,sAleS EXECUtIvE,1,NaN,"4294,0",11148,1,Y,NaN,12,"3,0",2,NaN,0,NaN,2,"3,0",7,NaN,0,7,"4294,0",1997,1000000000$,sAleS EXECUtIvE - Sales,NaN,1
1608,32,No,NaN,"498,0$",NaN,3,4,Medical,1,"966,0",3,1,93,3,2,MANUfaCtuRiNG DiReCtor,1,Married,"6725,0",13554,1,NaN,NaN,12,"3,0",3,NaN,1,"8,0",2,"4,0",8,NaN,6,3,"6725,0",1991,1000000000$,NaN,NaN,1
1609,36,Yes,travel_rarely,"530,0$",NaN,3,1,Life Sciences,1,"967,0",3,0,51,2,3,saLEs ExeCUTiVe,4,Married,"10325,0",5518,1,Y,NaN,11,NaN,1,NaN,1,NaN,6,"3,0",16,NaN,3,7,"10325,0",1987,1000000000$,NaN,NaN,0


In [12]:
# Restaurar el orden original 
df_unique = df_unique.sort_index()
# Remover el nombre del indice
df_unique.index.name = None 

In [13]:
df_unique['employeenumber'].duplicated().sum()

0

In [1]:
df_unique # Este es el df resultante que tiene las filas que queremos mantener en el df completo. 

NameError: name 'df_unique' is not defined

## Dirty code below ⬇️⬇️

In [ ]:
def change_commas(string):
    """
    Replaces commas with periods in a given string that represents a decimal number
    in international format (with commas as thousand separators and period as decimal separator).

    Note:
        If an error occurs during the replacement process (e.g., if the argument is not a string),
        the function will return np.nan (Not a Number) to indicate an invalid or unavailable value.
    """

    try:
        # Replace commas with periods in the string
        return float(string.replace(",", "."))
    
    except:
        # If an error occurs (e.g., if the argument is not a string),
        # return np.nan (Not a Number) to indicate an invalid or unavailable value.
        return np.nan


In [ ]:
def categorize_age(number):
    """
    Categorizes age into specific groups.

    This function takes an input number representing age and returns a specific category
    based on the age range.
    """
    if number >= 17 and number <= 25:
        return "Young Adults"
    
    elif number >= 26 and number <= 39:
        return "Young Adults"

    elif number >= 40 and number <= 59:
        return "Middle-aged"
    
    else:
        return "Older Adults"

# We have already created the function and verified that it works.
# The next step is to apply it to our entire DataFrame using the `apply()` method.
# This will return a Series, but we haven't stored this result in a variable.
# So, the next thing we'll do is create a new column in the DataFrame with the result of this apply.
df["age_category"] = df["age"].apply(categorize_age)
